In [11]:
import spacy
import re
import warnings
import ginza
import random
import numpy as np
from collections import defaultdict
from collections import deque
nlp = spacy.load('ja_ginza')

In [3]:
class Node:
    def __init__(self, id, token, parent):
        # id: 文中の語のid
        # word: 単語(lemma)
        # parent: 親ノードを表現．(id, <relation>)の．例えば(0, case)
        # isRoot: 根ノードか
        self.id = id
        self.token = token
        self.parent = parent
        self.child = []
        self.isRoot = (parent is None)

    def add_child(self, id, relation):
        # 子ノード 情報の追加．表記方法はparentと同じ．
        self.child.append((id, relation))

In [256]:
class SentenceTree: # 一つの文のみを処理する木
    def __init__(self, sentence, speaker):
        doc = nlp(sentence)
        self.id_lemma_dict = dict() # idとlemma
        self.node_list = [] # ノードを格納するリスト
        self.speaker = speaker # 発言者

        # ノードの登録
        for token in doc:
            self.id_lemma_dict[token.i] = token.lemma_
            idx = token.i
            lemma = token.lemma_

            if token.head.i == token.i: # ROOT
                parent = None
            else:
                parent = (token.head.i, token.dep_)
            self.node_list.append(Node(idx, lemma, parent))
        
        # 子ノード情報の登録
        for i in range(len(self.node_list)):
            if self.node_list[i].isRoot: # ROOTのインデックスを登録
                self.root_idx = i
            else: # ROOTでなければ親にアクセス
                (parent_id, relation) = self.node_list[i].parent
                self.node_list[parent_id].add_child(i, relation) # 子の追加
        # トークン数
        self.N = len(self.node_list)
        
        # 結果の確認
        """
        for i in range(self.N):
            print(self.node_list[i].id, self.node_list[i].token, self.node_list[i].child)
        """

    def bfs(self, token_list, start_idx=None, include_root=True):
        """
        input: token(捜索対象候補) start_idx(部分木の根,省略時はROOT) include_root(根も探索の対象か？)
        output: 存在すればid,しなければNone
        """
        if start_idx is None:
            start_idx = self.root_idx
        d = np.full(self.N, -1)
        q = deque()
        q.append(start_idx)
        d[start_idx] = 0

        if include_root and self.node_list[start_idx].token in token_list:
            return start_idx

        while len(q) != 0:
            n = q.popleft()
            for i in range(len(self.node_list[n].child)):
                child_id, _ =  self.node_list[n].child[i]
                if d[child_id] == -1:
                    q.append(child_id)
                    d[child_id] = d[n] + 1
                    if self.node_list[child_id].token in token_list:
                        return child_id
        return None

    def bfs_count(self, token_list, start_idx=None, include_root=True):
        """
        input: token(捜索対象候補) start_idx(部分木の根,省略時はROOT) include_root(根も探索の対象か？)
        output: 条件を満たす物の数を返す
        """ 
        ret = 0 
        if start_idx is None:
            start_idx = self.root_idx
        d = np.full(self.N, -1)
        q = deque()
        q.append(start_idx)
        d[start_idx] = 0

        if include_root and self.node_list[start_idx].token in token_list:
            ret += 1

        while len(q) != 0:
            n = q.popleft()
            for i in range(len(self.node_list[n].child)):
                child_id, _ =  self.node_list[n].child[i]
                if d[child_id] == -1:
                    q.append(child_id)
                    d[child_id] = d[n] + 1
                    if self.node_list[child_id].token in token_list:
                        ret += 1
        return ret
    
    def isVOTE(self):
        positive = 1
        not_target = "not VOTE"

        vote_word_list = ["投票", "吊る", "入れる", "する"]
        negative_aux_list = ["ぬ", "ない"]
        
        root = self.node_list[self.root_idx].token
        root_node = next(filter(lambda x: x.isRoot, self.node_list), None)
        
        child_list = []
        for i in range(len(root_node.child)):
            idx, relation = root_node.child[i]
            child_list.append(self.node_list[idx].token)
        print(child_list)
        
        child_vote = False
        for i in range(len(child_list)):
            if child_list[i] in vote_word_list:
                child_vote = True
        
        if root in vote_word_list or child_vote:
            if ("する" in self.id_lemma_dict.values()):
                if("投票" in self.id_lemma_dict.values()) == False:
                    return not_target + ": 投票を表す文章ではない"
                
            if root == "吊る":
                if self.root_idx == self.N-2:
                    return not_target + ": 投票を表す文章ではない"
                elif self.node_list[self.root_idx+1].token != "たい":
                    return not_target + ": 投票を表す文章ではない"
                    
            #人を探して、それを投票の対象と見なす
            target_id = self.bfs(["agent"])
            if target_id is None:
                return not_target + ": target発見できず"
            else:
                #"agent" "[" "03" "]"のように分かれて取得されるので、つなげて"Agent[03]"とする
                target = self.node_list[target_id].token + self.node_list[target_id+1].token +self.node_list[target_id+2].token + self.node_list[target_id+3].token

            for i in range(len(root_node.child)):
                idx, relation = root_node.child[i]
                if self.node_list[idx].token in negative_aux_list:
                    positive = 0
                    
            if root in negative_aux_list:
                positive = 0

            if positive:
                return(self.speaker, "VOTE", target, "POSITIVE")
            else:
                return(self.speaker, "VOTE", target, "NEGATIVE")
        
        else:
            return not_target + ": 投票を表す文章ではない"

In [257]:
sentence_list = [
    "Agent[03]に投票します。",
    "今日はAgent[03]に入れます。",
    "Agent[03]を吊りたいです。",
    "投票先はAgent[03]にします。",
    "Agent[03]には投票していないです。",
    "Agent[03]は吊りたくない。",
    "Agent[03]を吊ろうと思っている。",
]
dummy_list = [
    "Agent[03]を占った結果、白でした。",
    "Agent[03]は人間です。",
    "今日の占い結果は、Agent[03]が黒です。",
    "Agent[03]の占い結果は人狼です。",
    "Agent[03]は人狼ではありませんでした。",
    "Agent[03]が昨日怪しかったけど、村人でした。",
    "占ったところAgent[03]は人狼なんかじゃなかった。",
    "Agent[03]は黒という結果でした",
    "Agent[03]を占ったところ、人間と出ました",
    "Agent[03]が人狼だと思います。",
    "個人的にはAgent[03]は村人だと信じてる。",
    "今夜はAgent[03]を占いたい。",
    "占い師にはAgent[03]が村人か調べてほしい。",
    "Agent[03]が人狼か占ってほしい。"
]
sentence_list_sub = [
    "Agent[01]はAgent[03]に投票してほしい",
    "Agent[03]に投票してほしい。",
    "Agent[01]は誰に投票しましたか？",
    "Agent[01]は誰に投票しますか？"
    "DAY1にAgent[01]がAgent[02]に投票したので、Agent[01]に投票する",
    "Agent[02]はAgent[01]に投票した",
]
s1 = random.choice(sentence_list)
d1 = random.choice(dummy_list)
print("s1:"+s1)
print("d1:"+d1)
d2 = "Agent[03]は吊りたくない。"
s2 = "投票先はAgent[03]にします。"


s1:Agent[03]には投票していないです。
d1:Agent[03]の占い結果は人狼です。


In [258]:
doc = nlp(s2)
# 文節の係り受け解析
for span in ginza.bunsetu_spans(doc):
    for token in span.lefts:
        print(str(ginza.bunsetu_span(token))+' → '+str(span))
spacy.displacy.render(ginza.bunsetu_spans(doc), style="dep", options={"compact":True})

投票先は → します。
Agent → します。
[03]に → します。


In [253]:
def show_nlp(sentence):
    doc = nlp(sentence)
    for sent in doc.sents:
        print(sent)
        for token in doc:
            print(token.i, token.orth_, token.lemma_, token.pos_, token.tag_, token.dep_, token.head.i)
        print('EOS')
    spacy.displacy.render(doc, style="dep", options={"compact":True})

show_nlp(d2)

Agent[03]は吊りたくない。
0 Agent agent NOUN 名詞-普通名詞-一般 nsubj 5
1 [ ［ PUNCT 補助記号-括弧開 punct 0
2 03 03 NUM 名詞-数詞 compound 0
3 ] ］ PUNCT 補助記号-括弧閉 punct 0
4 は は ADP 助詞-係助詞 case 0
5 吊り 吊る NOUN 動詞-一般 advcl 7
6 たく たい AUX 助動詞 aux 5
7 ない ない ADJ 形容詞-非自立可能 ROOT 7
8 。 。 PUNCT 補助記号-句点 punct 7
EOS


In [252]:
test_tree = SentenceTree(d2, "Agent[02]")
print(test_tree.id_lemma_dict)
print(test_tree.node_list[test_tree.root_idx].token)

test_tree.isVOTE()

{0: 'agent', 1: '［', 2: '03', 3: '］', 4: 'は', 5: '吊る', 6: 'たい', 7: 'ない', 8: '。'}
ない
['吊る', '。']


('Agent[02]', 'VOTE', 'agent［03］', 'POSITIVE')

In [255]:
for i in range(len(sentence_list)):
    tree = SentenceTree(sentence_list[i], speaker="Agent[01]")
    print(sentence_list[i])
    print(tree.isVOTE(), "\n")
for i in range(len(dummy_list)):
    tree = SentenceTree(dummy_list[i], speaker="Agent[01]")
    print(dummy_list[i])
    print(tree.isVOTE(), "\n")

Agent[03]に投票します。
['agent', 'する', 'ます', '。']
('Agent[01]', 'VOTE', 'agent［03］', 'POSITIVE') 

今日はAgent[03]に入れます。
['今日', 'agent', '03', 'ます', '。']
('Agent[01]', 'VOTE', 'agent［03］', 'POSITIVE') 

Agent[03]を吊りたいです。
['agent', 'たい', 'です', '。']
('Agent[01]', 'VOTE', 'agent［03］', 'POSITIVE') 

投票先はAgent[03]にします。
['先', 'agent', '03', 'ます', '。']
('Agent[01]', 'VOTE', 'agent［03］', 'POSITIVE') 

Agent[03]には投票していないです。
['agent', 'する', 'て', 'ない', 'です', '。']
('Agent[01]', 'VOTE', 'agent［03］', 'NEGATIVE') 

Agent[03]は吊りたくない。
['吊る', '。']
('Agent[01]', 'VOTE', 'agent［03］', 'NEGATIVE') 

Agent[03]を吊ろうと思っている。
['吊る', 'て', '。']
('Agent[01]', 'VOTE', 'agent［03］', 'POSITIVE') 

Agent[03]を占った結果、白でした。
['結果', 'です', 'た', '。']
not VOTE: 投票を表す文章ではない 

Agent[03]は人間です。
['agent', 'です', '。']
not VOTE: 投票を表す文章ではない 

今日の占い結果は、Agent[03]が黒です。
['結果', 'agent', 'です', '。']
not VOTE: 投票を表す文章ではない 

Agent[03]の占い結果は人狼です。
['agent']
not VOTE: 投票を表す文章ではない 

Agent[03]は人狼ではありませんでした。
['agent', 'で', 'です', 'た', '。']
not VOTE: 投票を表す文章ではない 